In [ ]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt

In [4]:
# initialize data frames
df_day_ahead_energy_price = pd.read_csv('day_ahead_energy_price.csv')
df_generation_by_source = pd.read_csv('generation_by_source.csv')
df_historical_power_load = pd.read_csv('historical_power_load.csv')

In [ ]:
# head of day ahead energy price
df_day_ahead_energy_price.head()

In [ ]:
# Ensure the datetime column is in datetime format
df_day_ahead_energy_price['datetime_beginning_ept'] = pd.to_datetime(df_day_ahead_energy_price['datetime_beginning_ept'])

# Extract hour, day of the week, and month
df_day_ahead_energy_price['hour'] = df_day_ahead_energy_price['datetime_beginning_ept'].dt.hour
df_day_ahead_energy_price['day_of_week'] = df_day_ahead_energy_price['datetime_beginning_ept'].dt.dayofweek  # Monday=0, Sunday=6
df_day_ahead_energy_price['month'] = df_day_ahead_energy_price['datetime_beginning_ept'].dt.month

# Group by hour and calculate the mean system energy price
hourly_group = df_day_ahead_energy_price.groupby('hour')['system_energy_price_da'].mean()

# Plot the hourly cyclicality
hourly_group.plot(kind='line', title='Hourly Energy Price Cyclicality', ylabel='Average Energy Price', xlabel='Hour of the Day')
plt.show()

# Group by day of the week and calculate the mean system energy price
daily_group = df_day_ahead_energy_price.groupby('day_of_week')['system_energy_price_da'].mean()

# Plot the daily cyclicality
daily_group.plot(kind='bar', title='Daily Energy Price Cyclicality', ylabel='Average Energy Price', xlabel='Day of the Week')
plt.xticks(ticks=range(7), labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.show()

# Group by month and calculate the mean system energy price
monthly_group = df_day_ahead_energy_price.groupby('month')['system_energy_price_da'].mean()

# Plot the monthly cyclicality
monthly_group.plot(kind='bar', title='Monthly Energy Price Cyclicality', ylabel='Average Energy Price', xlabel='Month')
plt.xticks(ticks=range(1,13), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()


In [ ]:
# head of generation by source
df_generation_by_source.head()

In [ ]:
# Ensure the datetime column is in datetime format
df_generation_by_source['datetime_beginning_ept'] = pd.to_datetime(df_generation_by_source['datetime_beginning_ept'])

# Extract hour, day of the week, and month
df_generation_by_source['hour'] = df_generation_by_source['datetime_beginning_ept'].dt.hour
df_generation_by_source['day_of_week'] = df_generation_by_source['datetime_beginning_ept'].dt.dayofweek  # Monday=0, Sunday=6
df_generation_by_source['month'] = df_generation_by_source['datetime_beginning_ept'].dt.month

def plot_cyclicality(df, time_feature, value_column, title, ylabel, xlabel, xtick_labels=None):
    group = df.groupby(['fuel_type', time_feature])[value_column].mean().unstack()
    group.T.plot(kind='line', figsize=(10, 6), title=title, ylabel=ylabel, xlabel=xlabel)
    
    if xtick_labels:
        plt.xticks(ticks=range(len(xtick_labels)), labels=xtick_labels)
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()
    
# hourly cyclicality for each fuel type
plot_cyclicality(df_generation_by_source, 'hour', 'mw', 
                 title='Hourly Cyclicality by Fuel Type', 
                 ylabel='Average MW', xlabel='Hour of the Day')

# daily cyclicality for each fuel type
plot_cyclicality(df_generation_by_source, 'day_of_week', 'mw', 
                 title='Daily Cyclicality by Fuel Type', 
                 ylabel='Average MW', xlabel='Day of the Week', 
                 xtick_labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# monthly cyclicality for each fuel type
plot_cyclicality(df_generation_by_source, 'month', 'mw', 
                 title='Monthly Cyclicality by Fuel Type', 
                 ylabel='Average MW', xlabel='Month', 
                 xtick_labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])


# hourly cyclicality for each fuel type by percentage
plot_cyclicality(df_generation_by_source, 'hour', 'fuel_percentage_of_total', 
                 title='Hourly Cyclicality by Fuel Type (Percentage)', 
                 ylabel='Average Percentage of Total', xlabel='Hour of the Day')

# daily cyclicality for each fuel type by percentage
plot_cyclicality(df_generation_by_source, 'day_of_week', 'fuel_percentage_of_total', 
                 title='Daily Cyclicality by Fuel Type (Percentage)', 
                 ylabel='Average Percentage of Total', xlabel='Day of the Week', 
                 xtick_labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# monthly cyclicality for each fuel type by percentage
plot_cyclicality(df_generation_by_source, 'month', 'fuel_percentage_of_total', 
                 title='Monthly Cyclicality by Fuel Type (Percentage)', 
                 ylabel='Average Percentage of Total', xlabel='Month', 
                 xtick_labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [ ]:
# head of historical power load
df_historical_power_load.head()